Aim is to find out the sales of each product at a particular store

### Read Dataset

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

import pandas as pd
A = pd.read_csv("Desktop/DataSets/Bigmart Project/BigMart_Train.csv")

### Preview of the Dataset

In [2]:
A.shape

(8523, 12)

In [3]:
A.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
A.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

### EDA

#### Missing Data Treatment

In [6]:
A.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [7]:
for i in A.columns:
    if A[i].dtypes == "object":
        x = A[i].mode()[0]
        A[i] = A[i].fillna(x)
    else:
        x = A[i].mean()
        A[i] = A[i].fillna(x)

In [8]:
A.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

#### Skew

In [9]:
A.skew()  #skew is not more than 2, data is symmetrically distributed

Item_Weight                  0.090561
Item_Visibility              1.167091
Item_MRP                     0.127202
Outlet_Establishment_Year   -0.396641
Item_Outlet_Sales            1.177531
dtype: float64

In [10]:
A.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

### Defining  Y and removing statistically less important columns from A

In [11]:
Y = A[["Item_Outlet_Sales"]]
A = A.drop(labels=["Item_Identifier","Outlet_Identifier"],axis=1)

#### Separating Categorical and Continueous Variables and finding important features

In [12]:
cat = []
con = []

for i in A.columns:
    if A[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)

In [13]:
Q = A[con].corr()["Item_Outlet_Sales"].sort_values()
imp_con = list(Q[(Q < 1) & (Q > 0.3)].index)

In [14]:
imp_con

['Item_MRP']

In [15]:
#Dependancy between a Categorical and Continueous feature can be defined using ANOVA Analysis

def ANOVA(df,cat,con):
    from statsmodels.formula.api import ols
    relation = con + " ~ " + cat
    model = ols(relation,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    return round(anova_results.iloc[0,4],4)

imp_cat = []
Q = list(cat)
for i in Q :    
        print("--------------------")
        print("Item_Outlet_Sales vs",i)
        w = ANOVA(A,i,"Item_Outlet_Sales")
        print(w)
        if(w < 0.05):
            imp_cat.append(i) 

--------------------
Item_Outlet_Sales vs Item_Fat_Content
0.1412
--------------------
Item_Outlet_Sales vs Item_Type
0.0004
--------------------
Item_Outlet_Sales vs Outlet_Size
0.0
--------------------
Item_Outlet_Sales vs Outlet_Location_Type
0.0
--------------------
Item_Outlet_Sales vs Outlet_Type
0.0


In [16]:
imp_cat

['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

### Preprocessing

In [17]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X1 = pd.DataFrame(ss.fit_transform(A[imp_con]),columns=imp_con)

In [18]:
X2 = pd.get_dummies(A[imp_cat])

### FInal features to be used for Model

In [19]:
X = X2.join(X1)

In [20]:
X.head(3)

,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,...,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_MRP
0,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,1.747454
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,-1.489023
2,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0.010040


In [21]:
X.columns

Index(['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods', 'Outlet_Size_High',
       'Outlet_Size_Medium', 'Outlet_Size_Small',
       'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2',
       'Outlet_Location_Type_Tier 3', 'Outlet_Type_Grocery Store',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Supermarket Type3', 'Item_MRP'],
      dtype='object')

In [22]:
X.shape

(8523, 27)

# Split train test and Creating a model

In [23]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.2,random_state=21)

In [24]:
Q = ['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods', 'Outlet_Size_High',
       'Outlet_Size_Medium', 'Outlet_Size_Small',
       'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2',
       'Outlet_Location_Type_Tier 3', 'Outlet_Type_Grocery Store',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Supermarket Type3', 'Item_MRP']

Q[0:3]

['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast']

In [25]:
mse_l = []
for i in Q:
    j = Q[0:Q.index(i)+1]
    Xnew = X[j]
    print("\n==================================")
    print("\nImportant features: ",Xnew.columns)
    from sklearn.model_selection import train_test_split
    xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=23)
    
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    model = lr.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    
    from sklearn.metrics import mean_squared_error
    mse = round(mean_squared_error(ytest,pred),4)
    mse_l.append(mse)
    print(("Mean squared error is: ",mse))
    



Important features:  Index(['Item_Type_Baking Goods'], dtype='object')
('Mean squared error is: ', 2982956.2939)


Important features:  Index(['Item_Type_Baking Goods', 'Item_Type_Breads'], dtype='object')
('Mean squared error is: ', 2983230.4294)


Important features:  Index(['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast'], dtype='object')
('Mean squared error is: ', 2982946.1768)


Important features:  Index(['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned'],
      dtype='object')
('Mean squared error is: ', 2983972.534)


Important features:  Index(['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy'],
      dtype='object')
('Mean squared error is: ', 2983703.5191)


Important features:  Index(['Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy', 'Item_Type_Frozen Foods'],
      dtype='object'

In [26]:
min(mse_l)

1315826.0218

## Prediction on Test Data

Steps to perform on Testing dataset

1. B=read data
2. cols_to_drop from B with less statistical importance
3. replacing Missing values
4. skew if any
4. Preprocessing
5. Remove columns from Test data which are not present in the final selected featurees, If missing any columns add new columns to Bnew with value equal to 0
6. model and predict final Output

In [27]:
B = pd.read_csv("Desktop/DataSets/Bigmart Project/BigMart_Test.csv")

In [28]:
B.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.75,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.30,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1


In [29]:
B.shape

(5681, 11)

In [30]:
B.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [31]:
B = B.drop(labels=["Item_Identifier","Outlet_Identifier"],axis=1)

#### Missing Data Treatment

In [32]:
B.isna().sum()

Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [33]:
for i in B.columns:
    if B[i].dtypes == "object":
        x = B[i].mode()[0]
        B[i] = B[i].fillna(x)
    else:
        x = B[i].mean()
        B[i] = B[i].fillna(x)

In [34]:
B.isna().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

#### skew

In [35]:
B.skew()

Item_Weight                  0.142813
Item_Visibility              1.238312
Item_MRP                     0.136182
Outlet_Establishment_Year   -0.396306
dtype: float64

#### Preprocessing

In [42]:
cat = []
con = []

for i in B.columns:
    if B[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)
        
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
B1 = pd.DataFrame(ss.fit_transform(B[con]),columns=con)

B2 = pd.get_dummies(B[cat])

Bnew = B2.join(B1)

In [43]:
Bnew.head(2)

,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1.89746,-1.134077,-0.536555,0.139891
1,0,0,0,0,1,0,0,0,0,1,...,1,0,0,1,0,0,-1.03553,-0.531850,-0.868937,1.095512


In [44]:
Xnew.shape

(8523, 27)

In [45]:
Bnew.shape

(5681, 35)

#### Preparing Final input Columns

In [46]:
P = Xnew.columns

In [47]:
Q = list(Bnew.columns)

In [49]:
to_remove_from_Q = []

for columns in Q:
    if columns not in P:
        to_remove_from_Q.append(columns)
    else:
        pass
    
to_remove_from_Q

['Item_Fat_Content_LF',
 'Item_Fat_Content_Low Fat',
 'Item_Fat_Content_Regular',
 'Item_Fat_Content_low fat',
 'Item_Fat_Content_reg',
 'Item_Weight',
 'Item_Visibility',
 'Outlet_Establishment_Year']

In [50]:
Bfinal = Bnew.drop(labels=to_remove_from_Q, axis=1)

In [51]:
Bfinal.head()

,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,...,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_MRP
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,-0.536555
1,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,-0.868937
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,1.629848
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0.226697
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1.508110


In [52]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(Xnew,Y)
predicted = model.predict(Bfinal)

In [54]:
B1 = pd.read_csv("Desktop/DataSets/Bigmart Project/BigMart_Test.csv")

T = B1[["Outlet_Identifier"]]
T["Item_Outlet_Sales"] = predicted

In [55]:
T.head()

,Outlet_Identifier,Item_Outlet_Sales
0,OUT049,1776.0
1,OUT017,1442.0
2,OUT010,1896.0
3,OUT017,2532.0
4,OUT027,5148.0


In [56]:
T.to_csv("Desktop/DataSets/Bigmart Project/Sample_Submission.csv",index=None)

Properties of products and stores which play a key role in increasing sales are:
    
Item_Type, Item_MRP, Outlet_Size, Outlet_Location, Outlet_Type